In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-core langchain_openai python-dotenv langsmith pydantic spotipy

In [2]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
## Setup logging
import logging
import os
from dotenv import load_dotenv

load_dotenv(override=True)
logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)

# Define Search Tool

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=50,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)
name = search_tool.get_name()
desc = search_tool.description
desc

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

# Create ToolNode

In [5]:
from langgraph.prebuilt import ToolNode
from spotify_tools import get_playlists, create_spotify_playlist, get_track_list_from_playlist, add_tracks_to_playlist, filter_artists, get_artists_from_playlist, find_similar_artists, find_top_tracks
from plan import validate_plan

tools = [get_playlists, create_spotify_playlist, add_tracks_to_playlist, filter_artists, validate_plan, get_artists_from_playlist, search_tool, find_similar_artists, get_track_list_from_playlist, find_top_tracks]
# tools = [get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, search_tool]
tool_node = ToolNode(tools)

# Bind Tools to Model

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"), temperature=1.0)
# llm_with_tools = llm.bind_tools(tools, strict=True, parallel_tool_calls=False)
llm_with_tools = llm.bind_tools(tools, strict=True)

# System Prompt

In [7]:
from langchain_core.messages import SystemMessage
from prompts import Prompts

system_message = SystemMessage(Prompts.SYSTEM)


# First Message

In [8]:
from langchain_core.messages import HumanMessage
from prompts import Prompts

human_message = HumanMessage(Prompts.SPOTIFY)
ai_message = llm_with_tools.invoke([system_message, human_message])

2024-11-14 07:16:19,707 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [ ]:
if ai_message.tool_calls is None or ai_message.tool_calls[0]["name"] != "validate_plan":
    raise ValueError("should be tool message calling 'validate_plan'")

In [9]:
ai_message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  validate_plan (call_SXPqjUoSUkfz90z69igIE2Qb)
 Call ID: call_SXPqjUoSUkfz90z69igIE2Qb
  Args:
    plan: {'steps': [{'name': 'get-existing-artist-list', 'description': "Retrieve a list of unique artists from the 'New Rock and Blues' playlist.", 'success_criteria': 'Complete list of unique artists retrieved from the playlist.', 'tool': 'functions.get_artists_from_playlist', 'action': 'Tool Calling'}, {'name': 'find-similar-artists', 'description': "For each artist from the 'New Rock and Blues' playlist, find 3-4 artists of a similar style.", 'success_criteria': 'List of similar artists found for each artist in the playlist.', 'tool': 'functions.find_similar_artists', 'action': 'Tool Calling'}, {'name': 'filter-similar-artists', 'description': "Remove artists present in the 'New Rock and Blues' playlist from the list of similar artists.", 'success_criteria': "Filtered list of similar artists wit

In [10]:
plan = ai_message.tool_calls[0]["args"]
plan

{'plan': {'steps': [{'name': 'get-existing-artist-list',
    'description': "Retrieve a list of unique artists from the 'New Rock and Blues' playlist.",
    'success_criteria': 'Complete list of unique artists retrieved from the playlist.',
    'tool': 'functions.get_artists_from_playlist',
    'action': 'Tool Calling'},
   {'name': 'find-similar-artists',
    'description': "For each artist from the 'New Rock and Blues' playlist, find 3-4 artists of a similar style.",
    'success_criteria': 'List of similar artists found for each artist in the playlist.',
    'tool': 'functions.find_similar_artists',
    'action': 'Tool Calling'},
   {'name': 'filter-similar-artists',
    'description': "Remove artists present in the 'New Rock and Blues' playlist from the list of similar artists.",
    'success_criteria': "Filtered list of similar artists without any overlap with the original 'New Rock and Blues' playlist.",
    'tool': 'functions.filter_artists',
    'action': 'Tool Calling'},
   {'

# chat_prompt_template holds all messages (Human, AI, Tool)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt_template: ChatPromptTemplate = system_message + human_message + ai_message
messages = chat_prompt_template.format_messages()
messages

# Check for Tools in Message

In [ ]:
from langchain_core.messages import ToolMessage

tools_in_ai_message = []
tools_by_name = {tool.name: tool for tool in tools}
messages = chat_prompt_template.format_messages()
for tool_call in messages[-1].tool_calls:
    tool = tools_by_name[tool_call["name"]]
    tools_in_ai_message.append(tool)
    # observation = tool.invoke(tool_call["args"])
    # result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
if len(tools_in_ai_message) == 0:
    raise ValueError
else:
    print(tools_in_ai_message)

# Tool Call (should be validate_plan())

In [ ]:
response = tool_node.invoke({"messages": messages})
tool_message = response["messages"][0]
tool_message.pretty_print()

# Add Tool Response to Messages

In [ ]:
chat_prompt_template += tool_message
chat_prompt_template.format_messages()

# Send Messages with Tool result to LLM

In [ ]:
ai_tool_call_message = llm_with_tools.invoke(chat_prompt_template.format_messages())
ai_tool_call_message.pretty_print()

# Validate correct tool call

In [ ]:
if ai_tool_call_message.tool_calls is None or ai_tool_call_message.tool_calls[0]["name"] != "get_playlists":
    raise ValueError("should be tool message calling 'validate_plan'")

# Tool Call (should be get_playlists())

In [ ]:
chat_prompt_template += ai_tool_call_message
messages = chat_prompt_template.format_messages()
response = tool_node.invoke({"messages": messages})

In [ ]:
from langchain_core.messages import ToolMessage

playlist_tool_message: ToolMessage = response["messages"][0]
playlist_tool_message.pretty_print()

# Add Tool Response to Messages

In [ ]:
chat_prompt_template += playlist_tool_message
messages = chat_prompt_template.format_messages()

# Send Playlists Results to LLM

In [ ]:
ai_tool_call_message = llm_with_tools.invoke(messages)
ai_tool_call_message.pretty_print()

# Validate correct tool call

In [ ]:
if ai_tool_call_message.tool_calls is None or ai_tool_call_message.tool_calls[0]["name"] != "get_artists_from_playlist":
    raise ValueError("should be tool message calling 'validate_plan'")

# Add AI Tool Call to Messages

In [ ]:
chat_prompt_template += ai_tool_call_message
messages = chat_prompt_template.format_messages()

# Tool Call (get_artists_from_playlist())

In [ ]:
response = tool_node.invoke({"messages": messages})
track_list_tool_message: ToolMessage = response["messages"][0]

# Decode AI Tool Call message

In [ ]:
track_list_tool_message.pretty_print()

In [ ]:
# json.loads(track_list_tool_message.content)

# Add Tool Response to Messages

In [ ]:
chat_prompt_template += track_list_tool_message
messages = chat_prompt_template.format_messages()
messages

# Send Tool Result to LLM

In [ ]:
response = llm_with_tools.invoke(messages)

# AI Tool Call (find_similar_artist())

In [ ]:
if ai_tool_call_message.tool_calls is None or ai_tool_call_message.tool_calls[0]["name"] != "get_artists_from_playlist":
    raise ValueError("should be tool message calling 'validate_plan'")

In [ ]:
response.pretty_print()

In [ ]:
if ai_tool_call_message.tool_calls is None or ai_tool_call_message.tool_calls[0]["name"] != "find_similar_artists":
    raise ValueError("should be tool message calling 'validate_plan'")

# Add AI Tool Call to Messages

In [ ]:
chat_prompt_template += response
messages = chat_prompt_template.format_messages()
messages

# Tool Call (find_similar_artists())

In [ ]:
response = tool_node.invoke({"messages": messages})

In [ ]:
tool_message = response["messages"][0]
tool_message.pretty_print()

# Add Tool Response to Messages

In [ ]:
chat_prompt_template += tool_message
messages = chat_prompt_template.format_messages()
messages

# Send Tool Result to LLM

In [ ]:
response = llm_with_tools.invoke(messages)
response.pretty_print()

In [ ]:
chat_prompt_template += response
messages = chat_prompt_template.format_messages()
messages

In [ ]:
hm = HumanMessage("Why did you only asked for similar artists for a ubset of all artists in the playlist? How can the prompt be improved to make sure you go through all of them?")
chat_prompt_template += hm
response = llm_with_tools.invoke(messages)
response.pretty_print()

In [ ]:
filtered_artists: ToolMessage = response["messages"][0]
filtered_artists.pretty_print()

In [ ]:
response = llm_with_tools.invoke(messages)

In [ ]:
chat_prompt_template += response
chat_prompt_template.format_messages()

In [ ]:
response = tool_node.invoke({"messages": [response]})

In [ ]:
response